In [1]:
import sys


no_of_cpu = 4
max_cores = 8
executor_mem = '26g'



Job_Name = 'Nishant:Premium_Payment'


from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
#es_nodes = '10.35.12.5'
#es_nodes = '10.35.12.6'
es_nodes = '10.35.12.194'#,10.35.12.6,10.35.12.5
es_nodes_temp='10.35.12.194'
es_port = '5432'
es_user = ''
es_pwd = ''
mesos_ip = 'mesos://10.33.195.18:5050'#'mesos://10.35.12.5:5050'

spark.stop() #############NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL###############################

conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
#conf.set('spark.memory.fraction','.2')
conf.set('spark.executor.memory',executor_mem) 
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','600s')
conf.set('spark.sql.crossJoin.enabled', 'true')

conf.set('spark.ui.port','4052')

conf.set('spark.executor.heartbeatInterval','60s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx4g -Xms4g")

#conf.set("spark.shuffle.blockTransferService", "nio"); 
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70"); 
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer"); 
conf.set("spark.broadcast.compress", "true"); 
conf.set("spark.shuffle.compress", "true"); 
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.app.name", Job_Name)
#conf.set("spark.io.compression.codec","org.apache.spark.io.LZ4CompressionCodec");
#conf.set("spark.sql.inMemoryColumnarStorage.compressed", "true"); 
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','20g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
conf.set('spark.sql.shuffle.partitions','400')
#conf.set('spark.sql.crossJoin.enabled', 'true')
conf.set('es.nodes',es_nodes)
conf.set('es.port',es_port)
conf.set('es.nodes.wan.only','true')
conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

#conf.set('spark.es.net.http.auth.user','Spark')
#conf.set('spark.es.net.http.auth.pass','Jarkpet1Sap3')
conf.set('spark.num.executors','4')
conf.set('spark.debug.maxToStringFields', 200)

conf.set('spark.es.net.http.auth.user', es_user)
conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)

# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *


import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5462,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql.functions import hash
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import psycopg2
import time
from time import gmtime, strftime
import pytz
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2023-03-27 12:00:35.729709'

In [3]:

prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpspark"
pwd_prod="spark@456"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 

# Define the GPDB-Python connection options for PROD 
import psycopg2
import datetime
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:

def load_gpdb_jdbc(col_str,dbtable,dbschema=dbschema,col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .select(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .select(col_str).drop_duplicates()
   
    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def get_latest_progress():
    
    try: 
        gscPythonOptions = {
                 "url":prod_url ,
                 "user":user_prod ,
                 "password": pwd_prod,
                 "dbschema":"customermart",
                 "dbtable": "progress",#table change
                 "server.port":"1150-1170"} 
        
        # this query will fetch till what date we have inserted the records in target
        last_run= sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
            .select('table_name','source','to_datetime').filter(col('table_name')=='customer_activity')\
            .filter(col('source')=='activity3')
        
        Max_last_run =last_run.select(max('to_datetime')).first()[0]


        # if no record is available in progress then it will raise value error, in this case default value will be picked from except clause
        if Max_last_run is None:
            raise ValueError("No records found!")

        #print("Record Found. Progress updated till {}".format(Max_last_run))

    except Exception as E: 
        Max_last_run = datetime.datetime(2021, 5, 14, 0, 0)  # this is the default start date when no record is present in progress for customer_demographics table
        print("Executed_except",repr(E))
    return Max_last_run

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Generic function to save the progress from last run into Progress table
def update_progress(table_name,source,time_filter,records,start_time,starttime,status):
    output_index = "progress"
    schema = "customermart"


    import sys
    import time
    try:
        
                
#         df_progress_values = [{"table_name":table_name,"source":source,"from_datetime":time_filter['start_date'].strftime("%Y-%m-%d %H:%M:%S")
#                         ,"to_datetime":time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),'records':records,
#                          'start_time':start_time}]

        df_progress= sqlContext.createDataFrame([(table_name,source,
                                       time_filter['start_date'].strftime("%Y-%m-%d %H:%M:%S"),
                                         time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),records)]
                                     ,['table_name', 'source','from_datetime','to_datetime','records'])\
        .withColumn('start_time',lit(start_time)).withColumn('end_time',current_timestamp())\
                    .withColumn('executed_in_mins',lit((time.time() - starttime)/60))\
                                .withColumn('status',lit(status)).withColumn('remarks',lit(None))
        
#         schema = StructType([StructField("table_name", StringType(),True),StructField("source", StringType(),True),\
#                      StructField("from_datetime", StringType(),True),StructField("to_datetime", StringType(),True),\
#                      StructField("records", IntegerType(),True),StructField("start_time", StringType(),True)])

#         df_progress = sqlContext.createDataFrame(df_progress_values,schema)

        
        df_progress.write.format("greenplum")\
            .option("dbtable",output_index).option('dbschema',schema)\
            .option("server.port","1150-1170").option("url",prod_url)\
            .option("user", user_prod).option("password",pwd_prod).mode('append').save()
        
       
    except Exception as e:
        x = e
        print(x)
    else:
        x = 'success'
        print("Updated Progress for {table_name} from {source} until {to_datetime}. {records} records processed in this run".format(table_name=table_name, source=source,to_datetime=time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),records=records))
        print(x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Define the Generic update function which takes the records currently in temp table (created with prefix 1 to original customercoe table)
# The function would first try to insert the records in original table. It that fails then it will upsert the records
# Since there is no direct upsert query an update and insert query with where clause is used
# This funciton just runs the SQL queries in GPDB and does not use spark
def update_activity(conn_to,table_name,source,gpdb_spark_options = prod_gpdb_spark_options,
                  schema="customermart"):
    conn_to.rollback()
    primary_key = ['source_system_name','source_system_customer_id','activity_type','activity_id','commit_timestamp']
    update_fields = ['feature_value_1', 'feature_value_2', 'feature_string_1', 'feature_string_2',
               'feature_string_3', 'feature_string_4','feature_date_1']
    df_1 = sqlContext.read.format("greenplum").options(**gpdb_spark_options).option("dbschema",schema)\
    .option("dbtable","{}_3_staging".format(table_name)).load()
    out_columns = df_1.columns
    out_columns = [col for col in out_columns if "json" not in col]

    cur_to = conn_to.cursor()  

    only_insert_query = """insert into {schema}.{table_name} ({out_columns}) select {out_columns} from {schema}.{table_name}_3_staging""".format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns))
    update_query = """UPDATE {schema}.{table_name} orig
                          SET
                            {update_fields}
                          FROM
                            {schema}.{table_name}_3_staging temp
                           WHERE 
                            {primary_key}
                       """.format(schema=schema,table_name=table_name,primary_key= " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]),update_fields = " , ".join([ "{key} = temp.{key}".format(key=key) for key in update_fields]))
    insert_query = """ INSERT INTO {schema}.{table_name} ({out_columns})
                           SELECT {out_columns}
                           FROM
                             {schema}.{table_name}_3_staging temp
                           WHERE
                             NOT EXISTS (
                             SELECT 1 FROM {schema}.{table_name} orig WHERE 
                            {primary_key}
                            )
                        """.format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns),primary_key = " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]))

    try:
        print("Inside try segment")
        cur_to.execute(only_insert_query)
        print("Executed Only insert query")
#         update_progress(table_name,source,time_filter,records)
    except Exception as e:
        print("Inside except segment")
        conn_to.rollback()
        cur_to.execute(update_query)
        print("Executed update query")
        cur_to.execute(insert_query)
        print("Executed insert query")
#         update_progress(table_name,source,time_filter,records)
    conn_to.commit()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
latest_progress=get_latest_progress()
# dates in string format
str_d1 = latest_progress.strftime("%Y-%m-%d")
str_d2 = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d")

# convert string to date object
d1 = datetime.datetime.strptime(str_d1, "%Y-%m-%d")
d2 = datetime.datetime.strptime(str_d2, "%Y-%m-%d")

# difference between dates in timedelta
delta = d2 - d1
day=delta.days-2 #currently T-2 change 2 to 1 for T-1

time_filter={'start_date':latest_progress,'end_date':latest_progress+datetime.timedelta(days=day)}
time_filter

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'start_date': datetime.datetime(2023, 3, 7, 0, 0), 'end_date': datetime.datetime(2023, 3, 25, 0, 0)}

In [9]:
time_filter

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'start_date': datetime.datetime(2023, 3, 7, 0, 0), 'end_date': datetime.datetime(2023, 3, 25, 0, 0)}

In [10]:
col_str=["num_instrument_amt", "txt_instrument_no", "txt_payment_mode_cd", 'dat_instrument_date',
         'txt_payer_customer_id', 'num_instrument_id', 'tgt_commit_timestamp']

payment=load_gpdb_jdbc(col_str,"account_gc_acc_payment_entry",'public','tgt_commit_timestamp',time_filter)
payment=payment.filter(col("num_instrument_id").isNotNull()).filter(col("txt_payer_customer_id").isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
col_str=["num_proposal_no", "num_instrument_id"]

acc_map=load_gpdb_jdbc(col_str,"account_gc_acc_map_instrproposal",'public')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
col_str=["txt_modeofoperation", "num_reference_number", "txt_policy_no_char"]

gen_prop=load_gpdb_jdbc(col_str,"policy_gc_gen_prop_information_tab",'public').filter(col("txt_payer_customer_id").isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
col_str=["sum_insured", "num_reference_no"]

premium_register=load_gpdb_jdbc(col_str,"premium_register",'registers').filter(col("sum_insured").isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
## premium_register = premium_register.orderBy(col("sum_insured").desc()) 

In [14]:
df1=acc_map.join(gen_prop ,acc_map.num_proposal_no==gen_prop.num_reference_number,"left_outer")\
.drop(gen_prop.num_reference_number)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
inner=df1.join(premium_register ,df1.num_proposal_no==premium_register.num_reference_no,"left_outer")\
.drop(premium_register.num_reference_no)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
payment

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[num_instrument_amt: decimal(17,2), txt_instrument_no: string, txt_payment_mode_cd: string, dat_instrument_date: timestamp, txt_payer_customer_id: string, num_instrument_id: decimal(20,0), tgt_commit_timestamp: timestamp]

In [17]:
activity3 =payment.join(inner ,payment.num_instrument_id==inner.num_instrument_id,"left_outer")\
.drop(inner.num_instrument_id)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df_main=activity3.withColumn('activity_id', col('num_instrument_id')).withColumn('source_system_name', lit('GC'))\
.withColumn('activity_type', lit('Premium Payment'))\
.withColumn('feature_value_1', col('num_instrument_amt')*1.0)\
.withColumn('feature_string_3', col('txt_instrument_no'))\
.withColumn('feature_string_4', col('txt_payment_mode_cd'))\
.withColumn('feature_date_1', col('dat_instrument_date'))\
.withColumn('commit_timestamp', col('tgt_commit_timestamp'))\
.withColumn('source_system_customer_id', col('txt_payer_customer_id'))\
.withColumn('feature_string_1', col('txt_policy_no_char'))\
.withColumn('feature_string_2', col('txt_modeofoperation'))\
.withColumn('feature_value_2', col('sum_insured')*1.0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
## Updated the below window function (by adding : orderBY clause on feature_value_2)

from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1 = Window.partitionBy('source_system_customer_id','activity_id').orderBy(col("commit_timestamp").desc(),col("feature_value_2").desc())
df_main = df_main.withColumn('row_num',F.row_number().over(w1))
df_main = df_main.filter('row_num == 1')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df_main=df_main.select(col('source_system_name').cast('string'),col('source_system_customer_id').cast('string'),col("activity_type").cast('string'),col('activity_id').cast('string'),\
                         col('feature_value_1').cast(DoubleType()),col('feature_value_2').cast(DoubleType()),\
                         col('feature_string_1').cast('string'),col('feature_string_2').cast('string'),col('feature_string_3').cast('string'),col('feature_string_4').cast('string'),\
                         col('feature_date_1').cast("timestamp"), col('commit_timestamp').cast("timestamp"))
                         

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# len(s.dtypes)
df_main.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['source_system_name', 'source_system_customer_id', 'activity_type', 'activity_id', 'feature_value_1', 'feature_value_2', 'feature_string_1', 'feature_string_2', 'feature_string_3', 'feature_string_4', 'feature_date_1', 'commit_timestamp']

In [22]:
df_main.persist(pyspark.StorageLevel.MEMORY_AND_DISK)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_name: string, source_system_customer_id: string, activity_type: string, activity_id: string, feature_value_1: double, feature_value_2: double, feature_string_1: string, feature_string_2: string, feature_string_3: string, feature_string_4: string, feature_date_1: timestamp, commit_timestamp: timestamp]

In [23]:
df_main=df_main.repartition('source_system_customer_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
records=df_main.count() # 12:10--12:34-12:47
records

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2186491

In [25]:
output_index = "customer_activity_3_staging"
schema = "customermart"

import sys
try:
    
    df_main.write.format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()
    
    
except Exception as e:
    x = e
    print(x)
else:
    x = 200 #success
    print(x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

200

In [26]:
from pg import DB
db = DB(dbname="gpadmin", user='gpspark', passwd='spark@456', host='10.35.12.194', port= 5432)
TgtGCIDCount = db.query("select count(distinct source_system_customer_id) from customermart.customer_activity_3_staging").getresult()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
SrcGCIDCount=str(df_main.select(countDistinct("source_system_customer_id")).collect()[0][0])
# SrcSI=str(df_main.agg({'feature_value_2': 'sum'}).collect()[0][0])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# from pg import DB
# db = DB(dbname="gpadmin", user='gpspark', passwd='spark@456', host='10.35.12.194', port= 5432)
# TgtSI = db.query("select sum(feature_value_2) from customermart.customer_activity_3_staging").getresult()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
table_name='customer_activity'
source='activity3'
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
conn_to=conn_prod

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
update_activity(conn_to,table_name,source,prod_gpdb_spark_options, schema="customermart") # 12:48--12:53

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Inside try segment
Executed Only insert query

In [31]:
df_main.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['source_system_name', 'source_system_customer_id', 'activity_type', 'activity_id', 'feature_value_1', 'feature_value_2', 'feature_string_1', 'feature_string_2', 'feature_string_3', 'feature_string_4', 'feature_date_1', 'commit_timestamp']

In [32]:
gscPythonOptions = {
         "url": prod_url,
         "user": user_prod,
         "password": pwd_prod,
         "dbtable": "customermart.customer_activity",
         "server.port":"1150-1170"}


activity3_df = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
.select('source_system_name', 'source_system_customer_id', 'activity_type', 'activity_id', 'feature_value_1', 'feature_value_2', 'feature_string_1', 'feature_string_2', 'feature_string_3', 'feature_string_4', 'feature_date_1', 'commit_timestamp')\
.filter(col('activity_type')=='Premium Payment')\
.filter(col('commit_timestamp').between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd'))).dropDuplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
df_stag=df_main.select(['source_system_name','source_system_customer_id','activity_type','activity_id','commit_timestamp','feature_value_1', 'feature_value_2', 'feature_string_1', 'feature_string_2',
               'feature_string_3', 'feature_string_4','feature_date_1'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
activity3_df1=activity3_df.select(['source_system_name','source_system_customer_id','activity_type','activity_id','commit_timestamp','feature_value_1', 'feature_value_2', 'feature_string_1', 'feature_string_2',
               'feature_string_3', 'feature_string_4','feature_date_1'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
df_MissingData=df_stag.exceptAll(activity3_df1).withColumn('DataMisMatch',lit('Missing values at Target'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
reconresult=[{
    "SrcCount":SrcGCIDCount,
#     "SrcSI":SrcSI,
    "TargetCount":TgtGCIDCount[0][0],
#     "TgtSI":TgtSI[0][0],
    "TrgtMissMatchCount":df_MissingData.count()
}]

from pyspark.sql import Row
df_recon=sqlContext.createDataFrame(Row(**x) for x in reconresult)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
df_recon=df_recon.withColumn('Status',when((df_recon.SrcCount==df_recon.TargetCount) & (df_recon.TrgtMissMatchCount==0),'Success').otherwise('Failed'))
# df_recon=df_recon.withColumn('Status',when((df_recon.SrcSI==df_recon.TgtSI) & (df_recon.SrcCount==df_recon.TargetCount) & (df_recon.TrgtMissMatchCount==0),'Success').otherwise('Failed'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
status=df_recon.select(df_recon.Status).collect()[0][0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
status

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Success'

In [40]:
update_progress(table_name,source,time_filter,records,start_time,starttime,status) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Updated Progress for customer_activity from activity3 until 2023-03-25 00:00:00. 2186491 records processed in this run
success

In [41]:
df_main.unpersist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_name: string, source_system_customer_id: string, activity_type: string, activity_id: string, feature_value_1: double, feature_value_2: double, feature_string_1: string, feature_string_2: string, feature_string_3: string, feature_string_4: string, feature_date_1: timestamp, commit_timestamp: timestamp]

In [42]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…